In [ ]:
template = """\
# INSTRUCTION
- 당신은 긍/부정 분류기입니다.
- 각 대상 '만족도', '맛', '서비스', '가격'에 대한 평가가 긍정적인지 부정적인지를 분류하세요.
- 대상에 대한 평가가 없는 경우 '-'을 표시하세요.
- 예시를 보고 결과를 다음과 같은 딕셔너리 형식으로 출력하세요:

    "만족도": "긍정/부정/-",
    "맛": "긍정/부정/-",
    "서비스": "긍정/부정/-",
    "가격": "긍정/부정/-"

# SENTENCE: {sentence}
"""

In [90]:
from langchain_openai import ChatOpenAI 
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from operator import itemgetter

template = """\
# INSTRUCTION
- 당신은 {sentence}에서 {category}에 따라 긍/부정을 분류하는 역할입니다.
- {category}에 대한 반응을 [긍정, 부정, -] 중 하나로 분류하세요
- JSON 형식으로 출력하세요.
"""

prompt = PromptTemplate.from_template(template)
model = ChatOpenAI(model_name = "gpt-3.5-turbo")
output_parser = JsonOutputParser()
chain = prompt | model | output_parser

def make_runnable(category):
    return {"sentence": RunnablePassthrough() , "category": RunnablePassthrough(lambda _: category)}

combined = RunnableParallel(
    taste = make_runnable("맛") | chain,
    service = make_runnable("서비스") | chain,
    price = make_runnable("가격") | chain,
)

review = "직원분들 너무 친절하시고 음식맛은 대한민국 호텔부페 넘버원인데 말해뭐해 입니다 조금씩 일찍 입장 시켜주시는 융통성도 좋아요 쬐끔 아쉬운건 의외로 과일류 구색이 약하다는 거"

combined.invoke(review)


{'taste': {'feedback': '긍정'},
 'service': {'feedback': '긍정'},
 'price': {'feedback': '긍정'}}

In [109]:
from langchain_openai import ChatOpenAI 
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from operator import itemgetter

template = """\
# INSTRUCTION
- 당신은 SENTENCE에서 {category}에 따라 긍/부정을 분류하는 역할입니다.
- {category}에 대한 반응을 [긍정, 부정, -] 중 하나로 분류하세요
- 다음 KEYS를 참고하여 JSON 형식으로 출력하세요.

# KEYS:
- category <{category}반응과 관련된 대상>
- action <반응>

# SENTENCE: {sentence}
"""

prompt = PromptTemplate.from_template(template)
model = ChatOpenAI(model_name = "gpt-3.5-turbo")
output_parser = JsonOutputParser()
chain = prompt | model | output_parser

def make_runnable(text):
    return RunnablePassthrough.assign(category = lambda x: text)

combined = RunnableParallel(
    taste = make_runnable("맛") | chain,
    service = make_runnable("서비스") | chain,
    price = make_runnable("가격") | chain,
)

review = "직원분들 너무 친절하시고 음식맛은 대한민국 호텔부페 넘버원인데 말해뭐해 입니다 조금씩 일찍 입장 시켜주시는 융통성도 좋아요 쬐끔 아쉬운건 의외로 과일류 구색이 약하다는 거"

combined.invoke({"sentence": review})


{'taste': {'category': '맛', 'action': '긍정'},
 'service': {'category': '직원 및 음식', 'action': '긍정'},
 'price': {'category': '가격', 'action': '-'}}

In [94]:
cat = "맛"
add_runnable = RunnablePassthrough.assign(taste = lambda x: "안뇽")
add_runnable.invoke({"input": "다람쥐"})

{'input': '다람쥐', 'taste': '안뇽'}

In [101]:
cat = "맛"
add_runnable = RunnablePassthrough(lambda x: "안뇽")
add_runnable.invoke({"input": "다람쥐"})

{'input': '다람쥐'}